https://github.com/timesler/facenet-pytorch/blob/master/examples/infer.ipynb

In [10]:
ls

build/       dependencies/  facenet_pytorch.egg-info/  LICENSE.md  setup.py
codecov.yml  dist/          facenet-pytorch.ipynb      models/     tests/
data/        examples/      __init__.py                README.md


In [9]:
cd /content/drive/My Drive/Colab_Notebooks/facenet_pytorch

/content/drive/My Drive/Colab_Notebooks/facenet_pytorch


In [4]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing facenet_pytorch.egg-info/PKG-INFO
writing dependency_links to facenet_pytorch.egg-info/dependency_links.txt
writing requirements to facenet_pytorch.egg-info/requires.txt
writing top-level names to facenet_pytorch.egg-info/top_level.txt
package init file './models/__init__.py' not found (or not a regular file)
package init file './models/utils/__init__.py' not found (or not a regular file)
package init file './data/__init__.py' not found (or not a regular file)
writing manifest file 'facenet_pytorch.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/facenet_pytorch
copying build/lib/facenet_pytorch/__init__.py -> build/bdist.linux-x86_64/egg/facenet_pytorch
creating build/bdist.linux-x86_64/egg/facenet_pytorch/models
copying build/lib/facenet_pytorch/models/inception_resnet_v1.py -> build/bd

In [1]:
!pip install facenet-pytorch

     |████████████████████████████████| 1.9MB 2.6MB/s 


In [3]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


## MTCNN 모듈 정의

설명을 위해 기본 매개 변수가 표시되지만 필요하지는 않습니다. MTCNN은 신경망 및 기타 코드의 모음이므로 내부적으로 필요할 때 객체를 복사 할 수 있도록 장치를 다음과 같은 방식으로 전달해야합니다.

자세한 내용은 도움말 (MTCNN)을 참조하십시오.

In [5]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

## Inception Resnet V1 모듈 정의

사전 훈련 된 분류기의 경우 classify = True로 설정합니다. 이 예에서는 모델을 사용하여 임베딩 / CNN 기능을 출력합니다. 추론의 경우 모델을 평가 모드로 설정하는 것이 중요합니다.

자세한 내용은 help (InceptionResnetV1)를 참조하세요.

In [6]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

## Define a dataset and data loader

In [19]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('data/test_images')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
print(dataset.idx_to_class)
loader = DataLoader(dataset, collate_fn=collate_fn)


{0: 'angelina_jolie', 1: 'bradley_cooper', 2: 'kate_siegel', 3: 'paul_rudd', 4: 'shea_whigham'}


In [16]:
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

Face detected with probability: 0.999983
Face detected with probability: 0.999934
Face detected with probability: 0.999733
Face detected with probability: 0.999880
Face detected with probability: 0.999992


## 이미지 임베딩 계산
MTCNN은 모두 같은 크기의 얼굴 이미지를 반환하므로 Resnet 인식 모듈을 사용하여 쉽게 일괄 처리 할 수 있습니다. 여기에서는 이미지가 몇 개 밖에 없기 때문에 단일 배치를 구축하고 이에 대한 추론을 수행합니다.

실제 데이터 세트의 경우 특히 GPU에서 처리되는 경우 Resnet으로 전달되는 배치 크기를 제어하도록 코드를 수정해야합니다. 반복 테스트의 경우, 잘린 얼굴 또는 경계 상자의 계산을 한 번 수행하고 나중에 사용할 수 있도록 감지 된 얼굴을 저장할 수 있으므로 임베딩 또는 분류 (InceptionResnetV1 사용)에서 얼굴 감지 (MTCNN 사용)를 분리하는 것이 가장 좋습니다.

In [17]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

## Print distance matrix for classes

In [18]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists, columns=names, index=names))

                angelina_jolie  bradley_cooper  ...  paul_rudd  shea_whigham
angelina_jolie        0.000000        1.447480  ...   1.429847      1.399074
bradley_cooper        1.447480        0.000000  ...   1.013447      1.038684
kate_siegel           0.887728        1.313749  ...   1.388377      1.379655
paul_rudd             1.429847        1.013447  ...   0.000000      1.100503
shea_whigham          1.399074        1.038684  ...   1.100503      0.000000

[5 rows x 5 columns]
